 ## Export ONNX model

In [1]:
from ultralytics import YOLO

model = YOLO("../saves/yolo11n_osrs_21c.pt") # load with our trained weights

model.export(format="onnx",imgsz=320, device='cpu', nms=True) # input shape (1x3x320x320), target cpu, non-max sup attached

Ultralytics 8.3.140 🚀 Python-3.9.21 torch-2.6.0+cu118 CPU (AMD Ryzen 9 5900X 12-Core Processor)
YOLO11n summary (fused): 100 layers, 2,586,247 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from '../saves/yolo11n_osrs_21c.pt' with input shape (1, 3, 320, 320) BCHW and output shape(s) (1, 300, 6) (5.2 MB)
requirements: Ultralytics requirements ['onnxslim>=0.1.53', 'onnxruntime'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 0.9s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.52...
ONNX: export success ✅ 2.2s, saved as '../saves/yolo11n_osrs_21c.onnx' (10.0 MB)

Export complete (3.0s)
Results saved to /home/chitraz/Documents/Projects/RealTimeODinOSRS/saves
Predict:         yolo predict task=detect model=../saves/yolo11n_osrs_21c.onnx imgsz=320  
Validate:        yolo val task=detect model=../saves/yolo11n_osrs_21c.onnx imgsz=320 dat

'../saves/yolo11n_osrs_21c.onnx'

## Load ONNX model and run inference using onnxruntime

In [2]:
import onnxruntime 
import numpy as np
import cv2 as cv
import os

# onnx session
ort_sess = onnxruntime.InferenceSession('../saves/yolo11n_osrs_21c.onnx')

# create/pre-process a input batch 
root_path = os.path.join('/home/chitraz/Documents/Projects/RealTimeODinOSRS', 'dataset/val/images')
img_path = root_path + "/2922_3287_0_980_154_1945.png" # test image
input_img = cv.imread(img_path)
input_img = cv.resize(input_img, dsize=(320, 320), interpolation=cv.INTER_CUBIC) # resize to 320x320
input_img = np.transpose(input_img,(2,0,1)) # (HxWxC) -> (CxHxW) 
input_img = input_img.astype(np.float32)
input_img = np.expand_dims(input_img, 0) # add fake mini-batch bim

# inference 
outputs = ort_sess.run(None,{"images": input_img})[0][0]
outputs

array([[     157.42,         205,      180.18,      215.21,           1,           9],
       [    -10.524,      127.04,       53.52,      216.29,           1,           3],
       [     267.64,      121.71,      333.07,      200.07,           1,           3],
       ...,
       [     305.28,      123.64,      320.56,      135.96,     0.82692,          12],
       [     69.992,      130.72,      125.24,      139.77,     0.82687,          16],
       [     120.56,      31.437,      162.53,      82.665,     0.82664,           3]], dtype=float32)

In [3]:
class_filter = ['Cow','Cow_calf', 'Dairy_cow'
    ,'Tree','Evergreen_tree','Oak_tree','Willow_tree','Yew_tree'
    ,'Plant','Daisies','Thistle'
    ,'Rocks','Copper_rocks','Tin_rocks','Silver_rocks','Iron_rocks','Coal_rocks', 'Gold_rocks', 'Mithril_rocks', 'Adamantite_rocks', 'Clay_rocks']

CONF_TH = 0

for pred in outputs:
    box = pred[:3]
    conf = pred[4]
    class_id = pred[5]
    class_label = class_filter[int(class_id)]
    if conf > CONF_TH:
        print(pred)

[     157.42         205      180.18      215.21           1           9]
[    -10.524      127.04       53.52      216.29           1           3]
[     267.64      121.71      333.07      200.07           1           3]
[     127.02      100.21      185.67      113.88           1          10]
[     22.512      176.43      43.111      196.91           1          10]
[     299.94      300.05      318.81      318.82           1          10]
[     13.048      159.05      52.543      240.12           1           3]
[     16.323      143.16      42.648      160.41           1          10]
[     1.9739      220.92      71.823      272.41           1           3]
[     211.85      117.34      232.71      135.83           1          10]
[     41.849      218.22      123.58      279.76           1           3]
[     240.57       256.1      284.46      275.34           1           3]
[     85.331      219.86      162.17      233.22     0.99999          19]
[     136.47      131.75      158.62  

In [4]:
# clean up multi boxes for same object (NMS)

def nms(boxes, confs, IOU_TH  = 0.6):

    pass